Importación de librerías base para el problema 

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import sklearn as sk
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import plotly.express as px
import os

os.getcwd()


'c:\\Users\\andresfuentes\\LATAM'

Lectura de datos principal y creacion de copia auxiliar de de la data original, también se busco información extra sobre países de los destinos de vuelos y los días feriados de ese año para el origen de los vuelos

In [2]:
datos = pd.read_csv("dataset_SCL.csv", sep=",")
datos_original = datos.copy()
paises =  pd.read_excel("Prefijo Paises.xlsx", sheet_name= 'Hoja2')
feriados =  pd.read_excel("Feriados.xlsx")
datos.columns


C:\Users\andresfuentes\AppData\Local\Temp\ipykernel_8028\200450372.py:1: DtypeWarning: Columns (1,6) have mixed types. Specify dtype option on import or set low_memory=False.
  datos = pd.read_csv("dataset_SCL.csv", sep=",")


Index(['Fecha-I', 'Vlo-I', 'Ori-I', 'Des-I', 'Emp-I', 'Fecha-O', 'Vlo-O',
       'Ori-O', 'Des-O', 'Emp-O', 'DIA', 'MES', 'AÑO', 'DIANOM', 'TIPOVUELO',
       'OPERA', 'SIGLAORI', 'SIGLADES'],
      dtype='object')

verificación de los tipos de datos de este dataframe

In [3]:
datos.dtypes

Fecha-I      object
Vlo-I        object
Ori-I        object
Des-I        object
Emp-I        object
Fecha-O      object
Vlo-O        object
Ori-O        object
Des-O        object
Emp-O        object
DIA           int64
MES           int64
AÑO           int64
DIANOM       object
TIPOVUELO    object
OPERA        object
SIGLAORI     object
SIGLADES     object
dtype: object

transformanción de Fechas a formato tiempo

In [4]:
datos["Fecha-I"] = pd.to_datetime(datos["Fecha-I"])
datos["Fecha-O"] = pd.to_datetime(datos["Fecha-O"])
datos.dtypes

Fecha-I      datetime64[ns]
Vlo-I                object
Ori-I                object
Des-I                object
Emp-I                object
Fecha-O      datetime64[ns]
Vlo-O                object
Ori-O                object
Des-O                object
Emp-O                object
DIA                   int64
MES                   int64
AÑO                   int64
DIANOM               object
TIPOVUELO            object
OPERA                object
SIGLAORI             object
SIGLADES             object
dtype: object

creación de variables dif_min, hora_dia, hora, minutos

In [5]:
datos['dif_min'] = (datos["Fecha-O"]- datos["Fecha-I"])
datos['hora_dia'] = datos['Fecha-I'].dt.time
datos['hora'] = datos['Fecha-I'].dt.hour
datos['minutos'] = datos['Fecha-I'].dt.minute

datos.head()

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,...,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES,dif_min,hora_dia,hora,minutos
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,...,2017,Domingo,I,American Airlines,Santiago,Miami,0 days 00:03:00,23:30:00,23,30
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,...,2017,Lunes,I,American Airlines,Santiago,Miami,0 days 00:09:00,23:30:00,23,30
2,2017-01-03 23:30:00,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226,SCEL,KMIA,AAL,...,2017,Martes,I,American Airlines,Santiago,Miami,0 days 00:09:00,23:30:00,23,30
3,2017-01-04 23:30:00,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226,SCEL,KMIA,AAL,...,2017,Miercoles,I,American Airlines,Santiago,Miami,0 days 00:03:00,23:30:00,23,30
4,2017-01-05 23:30:00,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226,SCEL,KMIA,AAL,...,2017,Jueves,I,American Airlines,Santiago,Miami,-1 days +23:58:00,23:30:00,23,30


cambia la diferencia de las fechas de programación y salida de los vuelos a minutos.

In [6]:
datos["dif_min"] =  datos["dif_min"]/np.timedelta64(1,'m')

Homologación de variable Mes a nombre

In [7]:
conditions = [
    (datos['MES'] == 1),
    (datos['MES'] == 2),
    (datos['MES'] == 3),
    (datos['MES'] == 4),
    (datos['MES'] == 5),
    (datos['MES'] == 6),
    (datos['MES'] == 7),
    (datos['MES'] == 8),
    (datos['MES'] == 9),
    (datos['MES'] == 10),
    (datos['MES'] == 11),
    (datos['MES'] == 12)]
values = [
    'Enero',
    'Febrero',
    'Marzo',
    'Abril',
    'Mayo',
    'Junio',
    'Julio',
    'Agosto',
    'Septiembre',
    'Octubre',
    'Noviembre',
    'Diciembre']
datos['Mes_Nombre'] = np.select(conditions, values)
datos.head()

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,...,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES,dif_min,hora_dia,hora,minutos,Mes_Nombre
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,...,Domingo,I,American Airlines,Santiago,Miami,3.0,23:30:00,23,30,Enero
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,...,Lunes,I,American Airlines,Santiago,Miami,9.0,23:30:00,23,30,Enero
2,2017-01-03 23:30:00,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226,SCEL,KMIA,AAL,...,Martes,I,American Airlines,Santiago,Miami,9.0,23:30:00,23,30,Enero
3,2017-01-04 23:30:00,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226,SCEL,KMIA,AAL,...,Miercoles,I,American Airlines,Santiago,Miami,3.0,23:30:00,23,30,Enero
4,2017-01-05 23:30:00,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226,SCEL,KMIA,AAL,...,Jueves,I,American Airlines,Santiago,Miami,-2.0,23:30:00,23,30,Enero


Verificacion que el valores este correcto

In [8]:
datos[["dif_min","Fecha-O","Fecha-I"]]

,dif_min,Fecha-O,Fecha-I
0,3.0,2017-01-01 23:33:00,2017-01-01 23:30:00
1,9.0,2017-01-02 23:39:00,2017-01-02 23:30:00
2,9.0,2017-01-03 23:39:00,2017-01-03 23:30:00
3,3.0,2017-01-04 23:33:00,2017-01-04 23:30:00
4,-2.0,2017-01-05 23:28:00,2017-01-05 23:30:00
...,...,...,...
68201,46.0,2017-12-22 15:41:00,2017-12-22 14:55:00
68202,16.0,2017-12-25 15:11:00,2017-12-25 14:55:00
68203,40.0,2017-12-27 15:35:00,2017-12-27 14:55:00
68204,13.0,2017-12-29 15:08:00,2017-12-29 14:55:00


Creacion de variable temporada_alta y atraso_15


In [9]:
datos['temporada_alta'] = np.where(  ((datos['Fecha-I']>='2017-12-15') & (datos['Fecha-I']<='2018-03-03')) 
                                   | ((datos['Fecha-I']>='2017-07-15') & (datos['Fecha-I']<='2017-07-31'))
                                   | ((datos['Fecha-I']>='2017-01-01') & (datos['Fecha-I']<='2017-03-03'))
                                   | ((datos['Fecha-I']>='2017-09-11') & (datos['Fecha-I']<='2017-09-30'))
                                   ,1,0)
datos['atraso_15'] = np.where(datos['dif_min']>15,1,0)




creacion de variable periodo_dia

In [10]:
conditions = [
    (datos['hora'] >= 5) & (datos['hora'] < 12),
    (datos['hora'] >= 12) & (datos['hora'] < 19),
    (datos['hora'] >= 19) | ((datos['hora'] >= 0) & (datos['hora'] < 5))]
values = ['mañana', 'tarde', 'noche']
datos['periodo_dia'] = np.select(conditions, values)


exportacion de archivo csv con las variables creadas desde el dataframe

In [11]:
synthetic_features = datos[['temporada_alta','dif_min','atraso_15','periodo_dia']]

synthetic_features.to_csv("synthetic_features.csv", sep="|"
                          , header=True, index=False)

Calculo de Proporcion de atrasos de la base total

In [12]:
Prop_atrasos = datos.groupby(['atraso_15']
                         # 'Des-O'
                          #'OPERA',
                          #'MES',
                          #'DIA',
                          #'temporada_alta'
                        , dropna=False,as_index=False).agg(
    Cantidad_Vuelos = ('atraso_15','count'))
Prop_atrasos['Proporción de atrasos'] = (Prop_atrasos['Cantidad_Vuelos']/68203)*100


def f(row):
    if row == 0:
        val = 'No Atraso'
    else:
        val = 'Atraso'
    return val
Prop_atrasos['atraso_15'] = Prop_atrasos['atraso_15'].apply(f)
Prop_atrasos


Prop_atrasos

,atraso_15,Cantidad_Vuelos,Proporción de atrasos
0,No Atraso,55592,81.509611
1,Atraso,12614,18.494788


Grafico Proporción de atrasos en la data completa


In [13]:

fig = px.bar(Prop_atrasos
             , y ='atraso_15'
             , x ='Proporción de atrasos'
             , color_continuous_scale= 'Bluered'
             , color='Cantidad_Vuelos'
             , orientation= 'h'
             , title = 'Pproporción de atrasos'
             )
fig.show()

### Se observa que el 18.49% de los vuelos generales presentan atrasos

### Calculo de proporción de atrasos por Destino, se toma en cuenta para esta proporcion destinos mayores o iguales a 10 vuelos para la variable Destino de Operacion Des-O  

In [14]:
Destino_pre = datos.groupby(['Des-O']
                          #'OPERA',
                          #'MES',
                          #'DIA',
                          #'temporada_alta'
                        , dropna=False, as_index= False).agg(
    Cantidad_Vuelos = ('atraso_15','count'))


Destino_menores = Destino_pre[Destino_pre['Cantidad_Vuelos']<=10]

flex = Destino_menores['Des-O'].to_list() #agrega lista de destinos con menos de 10 vuelos
datos['Destino'] = np.where(datos['Des-O'].isin(flex), 'Otro',datos['Des-O']) #Creación de variable Destino donde se agrupan los DES-O con menos de 10 vuelos en categoria Otros


### Calculo de proporción de atrasos por Destino, se muestran el top 10 de Destinos con mayor proporción de atrasos

In [15]:
Destino = datos.groupby(['Destino']
                          #'OPERA',
                          #'MES',
                          #'DIA',
                          #'temporada_alta'
                        ,as_index=False, dropna=False).agg(
    Cantidad_Vuelos = ('atraso_15','count'),
    Cantidad_Atrasos = ('atraso_15','sum'))
                        
Destino['Proporción de atrasos'] = Destino['Cantidad_Atrasos']/Destino['Cantidad_Vuelos']*100 #Calculo proporción de atrasos
                       
Destino[Destino['Cantidad_Vuelos']>10].sort_values('Proporción de atrasos',ascending=False).head(10)
Destino_mayores = Destino[Destino['Cantidad_Vuelos']>10].sort_values('Proporción de atrasos',ascending=False)
Destino_mayores.sort_values('Proporción de atrasos',ascending=False).head(10)


,Destino,Cantidad_Vuelos,Cantidad_Atrasos,Proporción de atrasos
53,YSSY,194,113,58.247423
52,YMML,39,21,53.846154
17,Otro,25,10,40.000000
18,SAAR,136,54,39.705882
16,NZAA,361,135,37.396122
12,MDPC,54,18,33.333333
0,CYYZ,283,89,31.448763
20,SACO,1023,308,30.107527
46,SGAS,232,66,28.448276
27,SBGL,1435,405,28.222997


### grafíco de la proporcion de atrasos por Destino

In [16]:
fig = px.bar(Destino_mayores.head(10).sort_values('Proporción de atrasos',ascending=True)
             , y ='Destino'
             , x ='Proporción de atrasos'
             , color_continuous_scale= 'Bluered'
             , color='Cantidad_Atrasos'
             , orientation= 'h'
             , title = 'Ranking (Top 10) de la proporción de atrasos por destino segun cantidad de atrasos'
             )
fig.show()

### Se observa que de los Destinos de operación posibles el que presneta mayor proporcion de atrasos es el de YSSY  y YMML ambos en Australia, ¿es posible saber si Australia es el pais de destino con mayor proporción de atrasos?

 Para ellos usaremos los datos extraidos de  codificacion  Organización de Aviación Civil Internacional "OACI", La informacipon extraida de los paises tienen un prefijo para estos no todos con el mismo largo de caracteres, se hace una diferencia para poder juntar porteriomente.

In [17]:
paises['Des-O_2'] = paises['Código prefijo']
paises['Largo'] = paises['Des-O_2'].str.len()

paises_1 = paises[paises['Largo'] == 1]
paises_2 = paises[paises['Largo'] == 2]

def subs(x):
    return x[:2]
datos['Des-O_2'] = datos['Des-O'].apply(subs)
datos['Des-O_2']
paises_2 = paises_2.drop(columns=['Largo','Código prefijo'], axis=1)
paises_1 = paises_1.drop(columns=['Largo','Código prefijo'], axis=1)



#### Se crea  la variable País en la data completa

In [18]:

datos_test = datos.merge(paises_2, how='left', left_on='Des-O_2', right_on = 'Des-O_2')
datos_1 = datos_test[datos_test['País'].isnull()]
datos_2 = datos_test[datos_test['País'].notnull()]

datos_1 =  datos_1.drop(columns=[ 'País', ], axis=1)

def subs(x):
    return x[:1]
datos_1['Des-O_2'] = datos_1['Des-O'].apply(subs)
datos_1['Des-O_2']

datos_1 = datos_1.merge(paises_1, how='left', left_on='Des-O_2', right_on = 'Des-O_2')
datos_test = pd.concat([datos_1, datos_2], axis = 0)

datos_test =  datos_test.drop(columns=['LARGO_x','LARGO_y', 'LARGO'], axis=1)

datos_test =  datos_test.drop(columns=[ 'Des-O_2', ], axis=1)
datos_test


,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,...,dif_min,hora_dia,hora,minutos,Mes_Nombre,temporada_alta,atraso_15,periodo_dia,Destino,País
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,...,3.0,23:30:00,23,30,Enero,1,0,noche,KMIA,Estados contiguos de los Estados Unidos
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,...,9.0,23:30:00,23,30,Enero,1,0,noche,KMIA,Estados contiguos de los Estados Unidos
2,2017-01-03 23:30:00,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226,SCEL,KMIA,AAL,...,9.0,23:30:00,23,30,Enero,1,0,noche,KMIA,Estados contiguos de los Estados Unidos
3,2017-01-04 23:30:00,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226,SCEL,KMIA,AAL,...,3.0,23:30:00,23,30,Enero,1,0,noche,KMIA,Estados contiguos de los Estados Unidos
4,2017-01-05 23:30:00,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226,SCEL,KMIA,AAL,...,-2.0,23:30:00,23,30,Enero,1,0,noche,KMIA,Estados contiguos de los Estados Unidos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68201,2017-12-22 14:55:00,400,SCEL,SPJC,JAT,2017-12-22 15:41:00,400.0,SCEL,SPJC,JAT,...,46.0,14:55:00,14,55,Diciembre,1,1,tarde,SPJC,Perú
68202,2017-12-25 14:55:00,400,SCEL,SPJC,JAT,2017-12-25 15:11:00,400.0,SCEL,SPJC,JAT,...,16.0,14:55:00,14,55,Diciembre,1,1,tarde,SPJC,Perú
68203,2017-12-27 14:55:00,400,SCEL,SPJC,JAT,2017-12-27 15:35:00,400.0,SCEL,SPJC,JAT,...,40.0,14:55:00,14,55,Diciembre,1,1,tarde,SPJC,Perú
68204,2017-12-29 14:55:00,400,SCEL,SPJC,JAT,2017-12-29 15:08:00,400.0,SCEL,SPJC,JAT,...,13.0,14:55:00,14,55,Diciembre,1,0,tarde,SPJC,Perú


#### asignare datos_test a la data con los paises asignados para mantener la data original en caso de algun problema poder volver a ella.

In [19]:
Pais = datos_test.groupby([
                          #'Des-O'
                          #'OPERA',
                          #'MES',
                          #'DIA',
                          'País'
                        ], dropna=False, as_index=False).agg(
    Atraso_count = ('atraso_15','count'),
    Atraso_sum = ('atraso_15','sum')
                                            )

Pais['Proporción de atrasos'] = (Pais['Atraso_sum']/Pais['Atraso_count'])*100

Pais = Pais.sort_values('Proporción de atrasos',ascending=False)
Pais



,País,Atraso_count,Atraso_sum,Proporción de atrasos
1,Australia,233,134,57.510730
13,Nueva Zelanda,361,135,37.396122
18,República Dominicana,54,18,33.333333
4,Canadá,283,89,31.448763
15,Paraguay,232,66,28.448276
0,Argentina,9237,2452,26.545415
6,Colombia,1604,415,25.872818
3,"Brasil (también SD, SN, SS y SW)",5171,1316,25.449623
16,Perú,5271,1214,23.031683
19,Uruguay,1072,221,20.615672


#### Calculamos la proporcion de atraso por Pais de destino y podemos obersvar que efevtivamente Australia es el pais que presenta la mayor porocion de atrasos en los viajes. 

In [20]:
fig = px.bar(Pais.sort_values('Proporción de atrasos',ascending=True), y='País', x='Proporción de atrasos', color='Atraso_sum')
fig.show()

### Gráfico de proporcion de atrasos por país de Destino coloreado segun la suma de los atrasos. esto nos dice que la la proporcion de atrasos mayor la presenta Australia como país, pero la mayor cantidad de atrasos la tiene Chile

#### Analísis de proporción de atrasos segun tipo de vuelo, Internacional o Nacional.

In [21]:
Tipo_Vuelo = datos_test.groupby([
                          #'Des-O'
                          #'OPERA',
                          #'MES',
                          #'DIA',
                          'TIPOVUELO'
                        ], dropna=False, as_index=False).agg(
    Atraso_count = ('atraso_15','count'),
    Atraso_sum = ('atraso_15','sum')
                                            )

Tipo_Vuelo['Proporción de atrasos'] = (Tipo_Vuelo['Atraso_sum']/Tipo_Vuelo['Atraso_count'])*100




def f(row):
    if row == 'N':
        val = 'Nacional'
    else:
        val = 'Internacional'
    return val
    
Tipo_Vuelo['TIPOVUELO'] = Tipo_Vuelo['TIPOVUELO'].apply(f)
Tipo_Vuelo



,TIPOVUELO,Atraso_count,Atraso_sum,Proporción de atrasos
0,Internacional,31240,7048,22.560819
1,Nacional,36966,5566,15.057079


In [22]:
fig = px.bar(Tipo_Vuelo.sort_values('Proporción de atrasos',ascending=True), y = 'TIPOVUELO', x = 'Proporción de atrasos', color = 'Atraso_sum',text_auto=True)

fig.update_traces(textfont_size=8
                  , textangle=45
                  , textposition="inside"
                  ,marker_line_width=1
                  ,marker_line_color = 'black')

fig.show()

### Se observa que en los Vuelos Nacionales existe un 22.56% de proporción de atrasos vs un 15.05% de proporcion de atrasos en los vuelos internacionales.

#### Calculo de proporción de atrasos por Operador

In [23]:
Operador = datos.groupby([
                          #'Des-O'
                          'OPERA'
                          #'MES',
                          #'DIA',
                          #'temporada_alta'
                        ], dropna=False, as_index=False).agg(
    Cantidad_Vuelos = ('atraso_15','count'),
    Cantidad_Atrasos = ('atraso_15','sum'))
Operador['Proporción de atrasos'] = (Operador['Cantidad_Atrasos']/Operador['Cantidad_Vuelos'])*100
Operador = Operador.sort_values('Proporción de atrasos',ascending=False).head(7)
Operador


,OPERA,Cantidad_Vuelos,Cantidad_Atrasos,Proporción de atrasos
19,Plus Ultra Lineas Aereas,49,30,61.224490
20,Qantas Airways,195,113,57.948718
2,Air Canada,565,258,45.663717
17,Latin American Wings,1674,681,40.681004
11,Gol Trans,806,287,35.607940
7,Avianca,1152,306,26.562500
0,Aerolineas Argentinas,1949,468,24.012314


En esta sección observamos que el Operador "Plus Ultra Lineas Aereas" presenta la mayor porporcion de atrasos con 30 de sus 49 vuelos en total donde los sigue Qantas Airways con 113 de 195 y Air Canada con 258 de 565.

Grafico de proporción de atrasos por operador, coloreado por cantidad de atrasos 

In [24]:
fig = px.bar(Operador.sort_values('Proporción de atrasos',ascending=True), 
             y='OPERA', 
             x='Proporción de atrasos'
             , color_continuous_scale= 'Bluered'
             , color='Cantidad_Atrasos'
             ,title= 'Proporción de atrasos por operador segun cantidad de atrasos')
fig.show()

#### Observamos que los 3 Operadores que presentan mayor tasa de atrasos en sus vuelos son Plus Ultra Lineas Aereas, Qantas Airways y Air Canada 

ahora observaremos como se ve reflejada la tasa de atrasos por mes calculando de proporcion de atrasos por Mes

In [25]:
Mes = datos.groupby([
                          #'Des-O'
                          #'OPERA'
                          'Mes_Nombre'
                          #'DIA',
                          #'temporada_alta'
                        ], dropna=False, as_index=False).agg(
    Cantidad_Vuelos = ('atraso_15','count'),
    Cantidad_Atrasos = ('atraso_15','sum'))
Mes['Proporción de atrasos'] = (Mes['Cantidad_Atrasos']/Mes['Cantidad_Vuelos'])*100
Mes.sort_values('Proporción de atrasos',ascending=False)

,Mes_Nombre,Cantidad_Vuelos,Cantidad_Atrasos,Proporción de atrasos
5,Julio,5992,1758,29.339119
2,Diciembre,6356,1615,25.409062
10,Octubre,6032,1363,22.596154
9,Noviembre,6080,1199,19.720395
1,Agosto,5744,996,17.339833
11,Septiembre,5610,967,17.237077
6,Junio,4981,852,17.104999
3,Enero,6108,978,16.011788
8,Mayo,5240,827,15.782443
4,Febrero,5561,843,15.159144


Grafico de proporción de atrasos por mes coloreado por cantidad de atrasos 

In [26]:
fig = px.bar(Mes.sort_values('Proporción de atrasos'
                             ,ascending=True)
             , y='Mes_Nombre'
             , x='Proporción de atrasos'
             , color_continuous_scale= 'Bluered'
             , color='Cantidad_Atrasos'
             , title= 'proporción de atrasos por mes coloreado segun cantidad de atrasos'
             ,text_auto= True)
fig.show()


### Podemos observar que el mes que presenta un mayor proporcion de atrasos en sus vuelos es el mes de Julio con 29.33% de estos

### Estas cantidades en los atrasos del mes de Julio seran observables a nivel diaro, calcularemos la tasa de atrasos por dia segun la varable Fecha-I

In [27]:
datos['Fecha-I'] = datos['Fecha-I'].dt.normalize()

Fechas = datos.groupby(['Fecha-I']
                           #'OPERA',
                          #'MES',
                          #'DIA',
                          #,'temporada_alta'
                        , dropna=False, as_index= False).agg(
    Cantidad_Vuelos = ('atraso_15','count'),
    Cantidad_atrasos = ('atraso_15','sum'))
                    
Fechas['Proporción de atrasos'] = Fechas['Cantidad_atrasos']/Fechas['Cantidad_Vuelos']*100

fig = px.line(Fechas
              , x='Fecha-I'
              , y='Proporción de atrasos'
              ,markers = True)

fig.show()



Calculo de proporcion de atrasos por Dia de la semana

In [28]:
Dia = datos.groupby([
                          #'Des-O'
                          #'OPERA'
                          #'MES',
                          'DIANOM'
                          #'temporada_alta'
                        ],  dropna=False, as_index=False).agg(
    Cantidad_Vuelos = ('atraso_15','count'),
    Cantidad_Atrasos = ('atraso_15','sum'))
Dia['Proporción de atrasos'] = (Dia['Cantidad_Atrasos']/Dia['Cantidad_Vuelos'])*100
Dia.sort_values('Proporción de atrasos',ascending=False)

,DIANOM,Cantidad_Vuelos,Cantidad_Atrasos,Proporción de atrasos
6,Viernes,10292,2285,22.201710
2,Lunes,10131,2048,20.215181
1,Jueves,10251,2045,19.949273
4,Miercoles,9722,1661,17.084962
3,Martes,9662,1634,16.911613
5,Sabado,8352,1363,16.319444
0,Domingo,9796,1578,16.108616


In [29]:
fig = px.bar(Dia.sort_values('Proporción de atrasos',ascending=True)
             , y='DIANOM'
             , x='Proporción de atrasos'
             , color='Cantidad_Vuelos'
             , color_continuous_scale= 'Bluered'
             , title= 'Proporción de atrasos y cantidad de vuelos según día de la semama'
             , text_auto=True)
             
fig.update_traces(textfont_size=8
                  , textangle=45
                  , textposition="inside"
                  ,marker_line_width=1
                  ,marker_line_color = 'black')
fig.show()

### Observamos que los dias Viernes son los que presenetan mayor tasa de atrasos en canrtidad de vuelos

Calculo de proporcion de atraso por temporada

In [30]:
Temporada = datos.groupby([
                          #'Des-O'
                          #'OPERA',
                          #'MES',
                          #'DIA',
                          'temporada_alta'
                        ], dropna=False, as_index=False).agg(
    Cantidad_Vuelos = ('atraso_15','count'),
    Cantidad_Atrasos = ('atraso_15','sum'))

Temporada['Proporción de atrasos'] = (Temporada['Cantidad_Atrasos']/Temporada['Cantidad_Vuelos'])*100

Temporada.sort_values('Proporción de atrasos',ascending=False)

def f(row):
    if row == 0:
        val = 'Temporada Baja'
    else:
        val = 'Temporada Alta'
    return val
Temporada['temporada_alta'] = Temporada['temporada_alta'].apply(f)
Temporada


,temporada_alta,Cantidad_Vuelos,Cantidad_Atrasos,Proporción de atrasos
0,Temporada Baja,45995,8254,17.945429
1,Temporada Alta,22211,4360,19.629913


In [31]:
fig = px.bar(Temporada
             , y ='temporada_alta'
             , x ='Proporción de atrasos'
             , color='Cantidad_Atrasos'
             , color_continuous_scale= 'sunset'
             , range_color=[0,100]
             , orientation= 'h'
             , title = 'Cantidad de atrasos por temporada, coloreado por proporción de atrasos'
             )

fig.update_traces(textfont_size=8
                  , textangle=45
                  , textposition="inside"
                  ,marker_line_width=1
                  ,marker_line_color = 'black')
fig.show()

Observamos que para la temporada alta exite practicamente la mitad de vuelos con atrasos que en temporada alta, y el color parecido nos indica que la proporcion de vuelos con atrasos es muy similiar, por lo que podmeos decir que en temporada baja existen casi el doble de vuelos con atrasos, lo que hace que sean mas eficientes en las salidas de vuelos

In [32]:
datos_test['Fecha-I'] = datos_test['Fecha-I'].dt.normalize()
datos_test = datos_test.merge(feriados, how='left', left_on='Fecha-I', right_on = 'Fecha-I')
datos_test['Feriado'] = np.where(datos_test['Feriado'].isnull(),'No Feriado','Feriado')


In [33]:
Feriados = datos_test.groupby([
                          #'Des-O'
                          #'OPERA',
                          #'MES',
                          #'DIA',
                          'Feriado'
                        ], dropna=False, as_index=False).agg(
   Cantidad_Vuelos = ('atraso_15','count'),
    Cantidad_Atrasos = ('atraso_15','sum')
                                            )

Feriados['Proporción de atrasos'] = (Feriados['Cantidad_Atrasos']/Feriados['Cantidad_Vuelos'])*100

Feriados = Feriados.sort_values('Proporción de atrasos',ascending=False)
Feriados



,Feriado,Cantidad_Vuelos,Cantidad_Atrasos,Proporción de atrasos
1,No Feriado,64973,12114,18.644668
0,Feriado,3233,500,15.465512


In [34]:
fig = px.bar(Feriados.sort_values('Proporción de atrasos'
             ,ascending=True)
             , y='Feriado'
             , x='Proporción de atrasos'
             ,color_discrete_sequence=['steelblue'])
fig.show()


In [35]:

fig = px.histogram(datos, x="dif_min",
                   title='Histograma minutos de diferencia',
                   labels={'dif_min':'total bill'}, 
                   opacity=0.7,
                   color_discrete_sequence=['indianred'] 
                   )
fig.update_traces(marker_line_width = 1
                  ,marker_line_color = 'black')


fig.show()

Se crea un histograma para ver la distribucion de la ariable diferenxia de minutos enbtre Fecha_I y Fecha-O , variable dif_min.
nos muestra que exiten datos con diferenxia negativa, es decir que los vuelos partieron un poco antes de lo agendado.

In [36]:
atrasos = datos[datos['atraso_15']==1] #filtro para ver histograma solo de vuelos con atrasos


fig = px.histogram(atrasos
                   , x="dif_min"
                   ,title='Histograma minutos de diferencia'
                   ,labels={'dif_min':'total bill'}
                   ,opacity=0.8
                   ,color_discrete_sequence=['springgreen'] )

fig.update_traces(marker_line_width = 1
                  ,marker_line_color = 'black')
fig.show()

print(atrasos.describe())

                DIA           MES           AÑO       dif_min          hora  \
count  12614.000000  12614.000000  12614.000000  12614.000000  12614.000000   
mean      15.668384      7.239575   2017.000079     38.476058     13.180910   
std        8.472919      3.468836      0.008904     28.426395      5.746496   
min        1.000000      1.000000   2017.000000     16.000000      0.000000   
25%        9.000000      5.000000   2017.000000     20.000000      8.000000   
50%       15.000000      7.000000   2017.000000     27.000000     14.000000   
75%       23.000000     10.000000   2017.000000     45.000000     18.000000   
max       31.000000     12.000000   2018.000000    161.000000     23.000000   

            minutos  temporada_alta  atraso_15  
count  12614.000000    12614.000000    12614.0  
mean      26.317267        0.345648        1.0  
std       16.849035        0.475598        0.0  
min        0.000000        0.000000        1.0  
25%       14.000000        0.000000        

Observamos una distribucion muy smiliar a la exponencial, el cual su estimador para esta muestra es de  38.47 minutos de atraso.

# MODELACIÓN


## Prueba Random Forests

En esta seccion se entrenaran varios modelos para hacer una predicción de que un vuelo pueda sufrir un atraso o no.


In [37]:
datos_test.head()

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,...,hora_dia,hora,minutos,Mes_Nombre,temporada_alta,atraso_15,periodo_dia,Destino,País,Feriado
0,2017-01-01,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,...,23:30:00,23,30,Enero,1,0,noche,KMIA,Estados contiguos de los Estados Unidos,Feriado
1,2017-01-02,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,...,23:30:00,23,30,Enero,1,0,noche,KMIA,Estados contiguos de los Estados Unidos,No Feriado
2,2017-01-03,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226,SCEL,KMIA,AAL,...,23:30:00,23,30,Enero,1,0,noche,KMIA,Estados contiguos de los Estados Unidos,No Feriado
3,2017-01-04,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226,SCEL,KMIA,AAL,...,23:30:00,23,30,Enero,1,0,noche,KMIA,Estados contiguos de los Estados Unidos,No Feriado
4,2017-01-05,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226,SCEL,KMIA,AAL,...,23:30:00,23,30,Enero,1,0,noche,KMIA,Estados contiguos de los Estados Unidos,No Feriado


In [38]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report
from sklearn.feature_selection import SequentialFeatureSelector

from sklearn import preprocessing

le = preprocessing.LabelEncoder()

rf = RandomForestClassifier()
sfs = SequentialFeatureSelector(rf)

X = datos_test[['OPERA'
                ,'periodo_dia'
                ,'DIANOM'
                ,'Mes_Nombre'
                ,'temporada_alta'
                ,'Destino'
                ,'Feriado'
                ,'TIPOVUELO'
                ,'País']]
Y = datos_test['atraso_15']



### Las Variables Vlo-I, Ori-I, Vlo-O, Ori-O, SIGLAORI no se usaran en el modelo ya que presentan solo una categoria o tiene homologas con las de destino de operacion y sus codigos.


En la metodologia para seguir se elige la de usar label encoder tiempo en los procesamientos vs lo que es creacioón de variables dummy.

In [39]:
%%capture
X['OPERA'] = le.fit_transform(X['OPERA'])
X['DIANOM'] = le.fit_transform(X['DIANOM'])
X['periodo_dia'] = le.fit_transform(X['periodo_dia'])
X['Mes_Nombre'] = le.fit_transform(X['Mes_Nombre'])
X['temporada_alta'] = le.fit_transform(X['temporada_alta'])
X['Destino'] = le.fit_transform(X['Destino'])
X['Feriado'] = le.fit_transform(X['Feriado'])
X['TIPOVUELO'] = le.fit_transform(X['TIPOVUELO'])
X['País'] = le.fit_transform(X['País'])

Generacion de variables codificadas.

In [40]:

from sklearn.model_selection import train_test_split

# 80 % del conjunto de datos para entrenamiento y 20 % para validacion
X_train, X_test, Y_train, Y_test = train_test_split(X,
                                            Y,test_size=0.2, random_state= 666)

Primero se elegira un modelo Random Forest para hacer predicicones.

In [41]:
rf.fit(X_train,Y_train)


RandomForestClassifier()

Proceso de entrenamiento del modelo

In [42]:
rf.score(X_train,Y_train)

0.8604574444688806

Calculo de precicison del modelo en la muestra de entrenamiento es de 86.07%

In [43]:
Y_pred = rf.predict(X_test)


In [44]:
accuracy_score(Y_test, Y_pred)

0.7953379269901774

In [45]:
rf.fit(X_train,Y_train)

print(rf.score(X_train,Y_train))
print(rf.score(X_test,Y_test))
                                            
sfs.fit(X_test,Y_test)
sfs.get_feature_names_out().tolist()

0.8604574444688806
0.7968772907198358


['OPERA', 'periodo_dia', 'TIPOVUELO', 'País']

esta seccion se genero una seleccion de variables de pasos hacia adelante para seleccionar el conjunto de variables optimos para entrenar el modelo. mantendremos estas variables y generaremos un modleo de prediccion. 

In [46]:

X1 = datos_test[sfs.get_feature_names_out().tolist()]
Y1 = datos_test['atraso_15']

for i in sfs.get_feature_names_out().tolist():
    X1[i] = le.fit_transform(X1[i]) 
    print (i)

X1_train, X1_test, Y1_train, Y1_test = train_test_split(X1,
                                            Y1,test_size=0.2, random_state= 666 )





OPERA
periodo_dia
TIPOVUELO
País


C:\Users\andresfuentes\AppData\Local\Temp\ipykernel_8028\706619777.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [47]:
rf.fit(X1_train,Y1_train)

print(rf.score(X1_train,Y1_train))
print(rf.score(X1_test,Y1_test))


0.818433399310901
0.8218003225333529


In [48]:
from sklearn.model_selection import GridSearchCV

max_features_range = np.arange(1,10,1)
n_estimators_range = np.arange(1,10,1)
param_grid = dict(max_features = max_features_range, n_estimators = n_estimators_range)

grid = GridSearchCV(estimator = rf, param_grid = param_grid, cv=10)

In [49]:
%%capture
grid.fit(X1_train, Y1_train)

In [50]:
print("Mejor combinacion de hiper-parametros es  %s con precición de  %0.4f"
      % (grid.best_params_, grid.best_score_))

Mejor combinacion de hiper-parametros es  {'max_features': 2, 'n_estimators': 1} con precición de  0.8179


In [51]:
rf_final = RandomForestClassifier(max_features = grid.best_params_['max_features'], n_estimators =grid.best_params_['n_estimators'])
rf_final.fit(X1_train,Y1_train)
print(rf_final.score(X1_train,Y1_train))
print(rf_final.score(X1_test,Y1_test))


0.8183967451066637
0.8218003225333529


### Gradient Boosting

In [52]:
%%capture
import xgboost as xgb

modelo_xgb = xgb.XGBClassifier()

X = datos_test[['OPERA'
                ,'periodo_dia'
                ,'DIANOM'
                ,'Mes_Nombre'
                ,'temporada_alta'
                ,'Destino'
                ,'Feriado'
                ,'TIPOVUELO'
                ,'País']]
Y = datos_test[['atraso_15']]

X['OPERA'] = le.fit_transform(X['OPERA'])
X['DIANOM'] = le.fit_transform(X['DIANOM'])
X['periodo_dia'] = le.fit_transform(X['periodo_dia'])
X['Mes_Nombre'] = le.fit_transform(X['Mes_Nombre'])
X['temporada_alta'] = le.fit_transform(X['temporada_alta'])
X['Destino'] = le.fit_transform(X['Destino'])
X['Feriado'] = le.fit_transform(X['Feriado'])
X['TIPOVUELO'] = le.fit_transform(X['TIPOVUELO'])
X['País'] = le.fit_transform(X['País'])


In [53]:

X_train, X_test, Y_train, Y_test = train_test_split( X
                                                    ,Y
                                                    ,test_size=0.2
                                                    , random_state= 666)

In [54]:
%%capture
modelo_xgb.fit(X_train,Y_train)

In [55]:
modelo_xgb.score(X_train,Y_train)

0.8321603987977421

In [56]:

Y_pred = modelo_xgb.predict(X_test)


In [57]:
%%capture
sfs_modelo_xgb = SequentialFeatureSelector(modelo_xgb)

print(modelo_xgb.score(X_train,Y_train))
print(modelo_xgb.score(X_test,Y_test))
                                            
sfs_modelo_xgb.fit(X_test,Y_test)


In [58]:

sfs_modelo_xgb.get_feature_names_out().tolist()


['OPERA', 'periodo_dia', 'Mes_Nombre', 'temporada_alta']

In [59]:

X2 = X[sfs_modelo_xgb.get_feature_names_out().tolist()]
Y2 = Y['atraso_15']

for i in sfs_modelo_xgb.get_feature_names_out().tolist():
    X2[i] = le.fit_transform(X2[i]) 
    print (i)

X2_train, X2_test, Y2_train, Y2_test = train_test_split(X2,
                                            Y2,test_size=0.2, random_state= 666 )

OPERA
periodo_dia
Mes_Nombre
temporada_alta


C:\Users\andresfuentes\AppData\Local\Temp\ipykernel_8028\4293510424.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [61]:
modelo_xgb.fit(X2_train,Y2_train)

print(modelo_xgb.score(X2_train,Y2_train))
print(modelo_xgb.score(X2_test,Y2_test))

c:\Users\andresfuentes\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning:

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].

c:\Users\andresfuentes\Anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.



[10:55:57] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.8220804926325049
0.8226066559155549


In [62]:
from sklearn.model_selection import GridSearchCV

max_features_range = np.arange(1,10,1)
n_estimators_range = np.arange(1,10,1)
param_grid = dict(max_features = max_features_range, n_estimators = n_estimators_range)

grid = GridSearchCV(estimator = modelo_xgb, param_grid = param_grid, cv=10)

In [63]:
%%capture
grid.fit(X2_train, Y2_train)

In [64]:
print("Mejor combinacion de hiper-parametros es  %s con precición de  %0.4f"
      % (grid.best_params_, grid.best_score_))

Mejor combinacion de hiper-parametros es  {'max_features': 1, 'n_estimators': 1} con precición de  0.8163


In [65]:
modelo_xgb = xgb.XGBClassifier(max_features = grid.best_params_['max_features'], n_estimators =grid.best_params_['n_estimators'])
modelo_xgb.fit(X2_train,Y2_train)



[10:56:42] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:576: 
Parameters: { "max_features" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[10:56:42] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, max_features=1, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=1, n_jobs=12,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [66]:
print(modelo_xgb.score(X2_train,Y2_train))

print(accuracy_score(modelo_xgb.predict(X2_test),Y2_test))

0.8160692031375999
0.8201876557689488


### Árbol 

In [97]:
from sklearn.tree import DecisionTreeClassifier
modelo_arbol = DecisionTreeClassifier()

X = datos_test[['OPERA'
                ,'periodo_dia'
                ,'DIANOM'
                ,'Mes_Nombre'
                ,'temporada_alta'
                ,'Destino'
                ,'Feriado'
                ,'TIPOVUELO'
                ,'País']]
Y = datos_test[['atraso_15']]

X['OPERA'] = le.fit_transform(X['OPERA'])
X['DIANOM'] = le.fit_transform(X['DIANOM'])
X['periodo_dia'] = le.fit_transform(X['periodo_dia'])
X['Mes_Nombre'] = le.fit_transform(X['Mes_Nombre'])
X['temporada_alta'] = le.fit_transform(X['temporada_alta'])
X['Destino'] = le.fit_transform(X['Destino'])
X['Feriado'] = le.fit_transform(X['Feriado'])
X['TIPOVUELO'] = le.fit_transform(X['TIPOVUELO'])
X['País'] = le.fit_transform(X['País'])

C:\Users\andresfuentes\AppData\Local\Temp\ipykernel_8028\1681379917.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\andresfuentes\AppData\Local\Temp\ipykernel_8028\1681379917.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\andresfuentes\AppData\Local\Temp\ipykernel_8028\1681379917.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:/

In [123]:
X_train, X_test, Y_train, Y_test = train_test_split( X
                                                    ,Y
                                                    ,test_size=0.2
                                                )

In [124]:

%%capture
modelo_arbol.fit(X_train,Y_train)

modelo_arbol.score(X_train,Y_train)

Y_pred = modelo_arbol.predict(X_test)

In [129]:
%%capture
sfs_modelo_arbol = SequentialFeatureSelector(modelo_arbol)

In [130]:
print(modelo_arbol.score(X_train,Y_train))
print(modelo_arbol.score(X_test,Y_test))
                                            
sfs_modelo_arbol.fit(X_test,Y_test)


0.8605490799794736
0.7981234423105117


SequentialFeatureSelector(estimator=DecisionTreeClassifier())

In [131]:
sfs_modelo_arbol.get_feature_names_out().tolist()

['OPERA', 'Mes_Nombre', 'temporada_alta', 'TIPOVUELO']

In [132]:
X2 = X[sfs_modelo_arbol.get_feature_names_out().tolist()]
Y2 = Y['atraso_15']


In [135]:
for i in sfs_modelo_arbol.get_feature_names_out().tolist():
    X2[i] = le.fit_transform(X2[i]) 
    print (i)

X2_train, X2_test, Y2_train, Y2_test = train_test_split(X2,
                                            Y2,test_size=0.2)

modelo_arbol.fit(X2_train,Y2_train)

OPERA
Mes_Nombre
temporada_alta
TIPOVUELO


C:\Users\andresfuentes\AppData\Local\Temp\ipykernel_8028\1317182598.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



DecisionTreeClassifier()

In [136]:
print(modelo_arbol.score(X2_train,Y2_train))
print(modelo_arbol.score(X2_test,Y2_test))

0.8199728758888645
0.8176220495528514


In [142]:
from sklearn.model_selection import GridSearchCV

max_features_range = np.arange(1,10,1)
max_depth_range = np.arange(1,10,1)
param_grid = dict(max_features = max_features_range, max_depth = max_depth_range)

grid = GridSearchCV(estimator = modelo_arbol, param_grid = param_grid, cv=10)


grid.fit(X2_train, Y2_train)

print("Mejor combinacion de hiper-parametros es  %s con precición de  %0.4f"
      % (grid.best_params_, grid.best_score_))



c:\Users\andresfuentes\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning:

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].

c:\Users\andresfuentes\Anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.



[11:28:06] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:576: 
Parameters: { "max_features" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[11:28:06] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\Users\andresfuentes\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning:

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].

c:\Users\andresfuentes\Anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.

c:\Users\andresfuentes\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning:

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.

[11:28:06] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:576: 
Parameters: { "max_features" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[11:28:06] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:28:06] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:576: 
Parameters: { "max_features" } might not be used.

  This could be a false alarm, with some parameters getting used 

c:\Users\andresfuentes\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning:

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].

c:\Users\andresfuentes\Anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.

c:\Users\andresfuentes\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning:

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.

...

c:\Users\andresfuentes\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning:

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].

c:\Users\andresfuentes\Anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.

c:\Users\andresfuentes\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning:

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.

[11:28:13] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:576: 
Parameters: { "max_features" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[11:28:13] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:28:13] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:576: 
Parameters: { "max_features" } might not be used.

  This could be a false alarm, with some parameters getting used 

c:\Users\andresfuentes\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning:

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].

c:\Users\andresfuentes\Anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.

c:\Users\andresfuentes\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning:

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.

[11:28:13] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:576: 
Parameters: { "max_features" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[11:28:13] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:28:13] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:576: 
Parameters: { "max_features" } might not be used.

  This could be a false alarm, with some parameters getting used 

KeyboardInterrupt: 

# de aqui abajo estoy probando cosas


In [84]:
%%capture
from sklearn.linear_model import SGDClassifier

sgd = SGDClassifier()


X = datos_test[['OPERA'
                ,'periodo_dia'
                ,'DIANOM'
                ,'Mes_Nombre'
                ,'temporada_alta'
                ,'Destino'
                ,'Feriado'
                ,'TIPOVUELO'
                ,'País']]
Y = datos_test['atraso_15']

X['OPERA'] = le.fit_transform(X['OPERA'])
X['periodo_dia'] = le.fit_transform(X['periodo_dia'])
X['DIANOM'] = le.fit_transform(X['DIANOM'])
X['Mes_Nombre'] = le.fit_transform(X['Mes_Nombre'])
X['temporada_alta'] = le.fit_transform(X['temporada_alta'])
X['Destino'] = le.fit_transform(X['Destino'])
X['Feriado'] = le.fit_transform(X['Feriado'])
X['TIPOVUELO'] = le.fit_transform(X['TIPOVUELO'])
X['País'] = le.fit_transform(X['País'])


X3_train, X3_test, Y3_train, Y3_test = train_test_split(X,
                                            Y,test_size=0.2, random_state = 666)

In [85]:
sgd.fit(X3_train,Y3_train)
print(sgd.score(X3_test,Y3_test)) 
print(sgd.score(X3_train,Y3_train))


0.8180618677613253
0.814309801334213


In [87]:
modelo_sgd = SGDClassifier()

In [90]:
%%capture
X3 = datos_test[['OPERA', 'Feriado', 'TIPOVUELO', 'País']]
Y3 = datos_test['atraso_15']
X3['OPERA'] = le.fit_transform(X['OPERA'])
X3['Feriado'] = le.fit_transform(X['Feriado'])
X3['TIPOVUELO'] = le.fit_transform(X['TIPOVUELO'])
X3['País'] = le.fit_transform(X['País'])
X3_train, X3_test, Y3_train, Y3_test = train_test_split(X,
                                            Y,test_size=0.2)



In [92]:
modelo_sgd.fit(X3_train,Y3_train)

print(modelo_sgd.score(X3_train,Y3_train))
print(modelo_sgd.score(X3_test,Y3_test))


0.8134850817388755
0.8213605043248791


In [ ]:
from sklearn.model_selection import GridSearchCV
import numpy as np

max_features_range = np.arange(1,10,1)
n_estimators_range = np.arange(1,10,1)
param_grid = dict(max_features = max_features_range, n_estimators = n_estimators_range)

grid = GridSearchCV(estimator = gb, param_grid = param_grid, cv=10)

grid.fit(X_train, Y_train)
print("Mejor combinacion de hiper-parametros es  %s con precición de  %0.4f"
      % (grid.best_params_, grid.best_score_))

Mejor combinacion de hiper-parametros es  {'max_features': 1, 'n_estimators': 1} con precición de  0.8143


In [ ]:
%%capture
import xgboost 

xgb = xgboost.XGBClassifier()


X = datos_test[['OPERA'
                ,'periodo_dia'
                ,'DIANOM'
                ,'Mes_Nombre'
                ,'temporada_alta'
                ,'Destino'
                ,'Feriado'
                ,'TIPOVUELO'
                ,'País']]
Y = datos_test['atraso_15']

X['OPERA'] = le.fit_transform(X['OPERA'])
X['periodo_dia'] = le.fit_transform(X['periodo_dia'])
X['DIANOM'] = le.fit_transform(X['DIANOM'])
X['Mes_Nombre'] = le.fit_transform(X['Mes_Nombre'])
X['temporada_alta'] = le.fit_transform(X['temporada_alta'])
X['Destino'] = le.fit_transform(X['Destino'])
X['Feriado'] = le.fit_transform(X['Feriado'])
X['TIPOVUELO'] = le.fit_transform(X['TIPOVUELO'])
X['País'] = le.fit_transform(X['País'])


X_train, X_test, Y_train, Y_test = train_test_split(X,
                                            Y,test_size=0.2)



In [ ]:
xgb.fit(X_train,Y_train)

print(xgb.score(X_train,Y_train))
print(xgb.score(X_test,Y_test))
                                            
sfs.fit(X_test,Y_test)
sfs.get_feature_names_out()

c:\Users\andresfuentes\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning:

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].

c:\Users\andresfuentes\Anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.



[13:06:04] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.8326735576570633
0.8228265650197918


array(['OPERA', 'Feriado', 'TIPOVUELO', 'País'], dtype=object)

In [ ]:
%%capture

X1 = datos_test[['OPERA', 'periodo_dia', 'TIPOVUELO', 'País']]
Y1 = datos_test['atraso_15']
X1['OPERA'] = le.fit_transform(X['OPERA'])
X1['periodo_dia'] = le.fit_transform(X['periodo_dia'])
X1['TIPOVUELO'] = le.fit_transform(X['TIPOVUELO'])
X1['País'] = le.fit_transform(X['País'])
X1_train, X1_test, Y1_train, Y1_test = train_test_split(X1,
                                            Y1,test_size=0.2)


In [ ]:

xgb.fit(X_train,Y_train)

print(rf.score(X_train,Y_train))
print(rf.score(X_test,Y_test))

c:\Users\andresfuentes\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning:

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].

c:\Users\andresfuentes\Anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.



[13:06:50] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\Users\andresfuentes\Anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning:

The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- DIANOM
- Destino
- Feriado
- Mes_Nombre
- TIPOVUELO
Feature names must be in the same order as they were in fit.




ValueError: X has 9 features, but RandomForestClassifier is expecting 4 features as input.

In [ ]:
%%capture
from sklearn.tree import DecisionTreeClassifier
arbol = DecisionTreeClassifier()

X = datos_test[['OPERA'
                ,'periodo_dia'
                ,'DIANOM'
                ,'Mes_Nombre'
                ,'temporada_alta'
                ,'Destino'
                ,'Feriado'
                ,'TIPOVUELO'
                ,'País']]
Y = datos_test['atraso_15']

X['OPERA'] = le.fit_transform(X['OPERA'])
X['periodo_dia'] = le.fit_transform(X['periodo_dia'])
X['DIANOM'] = le.fit_transform(X['DIANOM'])
X['Mes_Nombre'] = le.fit_transform(X['Mes_Nombre'])
X['temporada_alta'] = le.fit_transform(X['temporada_alta'])
X['Destino'] = le.fit_transform(X['Destino'])
X['Feriado'] = le.fit_transform(X['Feriado'])
X['TIPOVUELO'] = le.fit_transform(X['TIPOVUELO'])
X['País'] = le.fit_transform(X['País'])


X_train, X_test, Y_train, Y_test = train_test_split(X,
                                            Y,test_size=0.2)



In [ ]:
arbol.fit(X_train,Y_train)

print(arbol.score(X_train,Y_train))
print(arbol.score(X_test,Y_test))
                                            
sfs.fit(X_test,Y_test)
sfs.get_feature_names_out()

0.8611722014515065
0.7957044421639056


array(['OPERA', 'periodo_dia', 'TIPOVUELO', 'País'], dtype=object)

In [ ]:
%%capture
X1 = datos_test[['OPERA', 'periodo_dia', 'TIPOVUELO', 'País']]
Y1 = datos_test['atraso_15']
X1['OPERA'] = le.fit_transform(X['OPERA'])
X1['periodo_dia'] = le.fit_transform(X['periodo_dia'])
X1['TIPOVUELO'] = le.fit_transform(X['TIPOVUELO'])
X1['País'] = le.fit_transform(X['País'])
X1_train, X1_test, Y1_train, Y1_test = train_test_split(X1,
                                            Y1,test_size=0.2)



In [ ]:
arbol.fit(X_train,Y_train)

print(arbol.score(X_train,Y_train))
print(arbol.score(X_test,Y_test))

0.8611722014515065
0.7952646239554317


In [ ]:
importances = rf.feature_importances_

In [ ]:

forest_importances = pd.Series(importances)

sorted_indices = np.argsort(importances)[::-1]
 
feat_labels = X_train.columns
 
for f in range(X_train.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30,
                            feat_labels[sorted_indices[f]],
                            importances[sorted_indices[f]]))

 1) DIANOM                         0.249848
 2) Mes_Nombre                     0.245421
 3) Destino                        0.238418
 4) OPERA                          0.095233
 5) temporada_alta                 0.054417
 6) País                           0.044851
 7) periodo_dia                    0.035722
 8) Feriado                        0.032167
 9) TIPOVUELO                      0.003922


In [ ]:
import streamlit as st
st.set_page_config(page_title="Vuelos",
                    page_icon= ":bar_chart:",
                    layout= 'wide')